In [1]:
from mqtt import MQTT_connection
from IPython.display import display
import ipywidgets as widgets
import time, threading
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import glob



In [2]:
%matplotlib widget


In [3]:
class Controller:
    def __init__(self):
        self.mqtt = MQTT_connection("192.168.1.3")
        self.button = widgets.Button(description = "Switch Light Status")
        self.light_state = 0
        self.temperatures = []
        self.datetimes = []
        self.plot_data ={"time":range(len(self.temperatures)),"temperature":self.temperatures}        
        self.button.on_click(self.switch_lights)
        
        self.toggle = widgets.ToggleButton(description = "Switch Lights", value = bool(self.mqtt.light_state), 
                                            button_style = "info")
        self.toggle.observe(self.switch_lights, "value")
        display(self.toggle)
        
    def start_display(self):
        self.update_thread = threading.Thread(target = self.update_temperatures, args = ())
        self.update_thread.start()
   
    def switch_lights(self, x):
        #x passed because of how the ipywidgets button passes stuff to the connected function
        self.mqtt.switch_lights()
               
    
    def update_temperatures(self):
        
        fig, ax = plt.subplots()
        ax.autoscale(enable = True, axis = "both", tight = True)
        ax.set_xlabel("Time")
        ax.set_ylabel("Temperature (C)")
        ax.set_title(f"OptoWorld Live State")
        light_indicator = plt.Circle([0.8,1], radius = 0.01)
        ax.add_artist(light_indicator)
        line, = ax.plot([],[])
        plt.show()
        self.alive = True
        counter = 0
        while self.alive:
            self.temperatures.append(self.mqtt.q.get())
            self.mqtt.q.task_done()
            self.datetimes.append(datetime.datetime.now())
            
            
            counter +=1

            x_data = range(len(self.temperatures))
            line.set_xdata(self.datetimes)
            line.set_ydata(self.temperatures)
            light_indicator.center = [self.datetimes[-1] - datetime.timedelta(seconds = 30), 25]
#                 self.ax.relim()
#                 self.ax.autoscale_view(tight = True)
            ax.set_xlim([self.datetimes[0],self.datetimes[-1]+datetime.timedelta(seconds = 5)])
            ax.set_ylim(ymin = np.min(self.temperatures) -1, ymax = np.max(self.temperatures)+1)
            fig.canvas.draw()
            time.sleep(0.5)
                                        
            
            
            
            

In [4]:
c = Controller()

ToggleButton(value=False, button_style='info', description='Switch Lights')

In [5]:
c.start_display()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def map_values(x, in_min, in_max, out_min, out_max):
    return (x-in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                  # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = np.array(ia[1:] != ia[:-1])     # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

In [7]:
widgets.interact_manual.opts["manual_name"] = "Plot"
@widgets.interact_manual(description ="update")
def plot_temperatures(file_to_plot = glob.glob("./*.txt"), 
                      time_range = widgets.IntRangeSlider(description = "range to plot", value = [0,100], min = 0, max = 100)):
#     plt.close("all")
    fig1, axes = plt.subplots(figsize = (10,5))
    file = pd.read_csv(file_to_plot, sep = "\t")
    time_range = map_values(np.array([time_range[0], time_range[1]]), 0, 100, 0, len(file))
    lower, upper = time_range.astype(np.int32)
    file = file.loc[lower:upper]
    
    min_temp = file["temperature"].min()
    max_temp = file["temperature"].max()
    lengths, starts, values =rle(file["lightstate"])
    for length, start, value in zip(lengths, starts, values):
        if value == 1:
            print(length, start)
            x = mdates.date2num(pd.to_datetime(file["time"].iloc[start]))
            y = min_temp -1
            try:
                l = mdates.date2num(pd.to_datetime(file["time"].iloc[start+length])) - x
            except:
                l = mdates.date2num(pd.to_datetime(file["time"].iloc[-1])) - x
            h = max_temp - min_temp + 2
            rect = plt.Rectangle((x,y), l, h, facecolor = "lightblue", alpha = 0.3)
            axes.add_artist(rect)    
    axes.plot(pd.to_datetime(file["time"]), file["temperature"])
    fig1.autofmt_xdate()
    

interactive(children=(Dropdown(description='file_to_plot', options=('./testfile_old.txt', './testfile.txt'), v…

In [8]:
widgets.interact_manual.opts["manual_name"] = "Download file"

@widgets.interact_manual
def dowload_file(filename = glob.glob("./*.txt")):
    js_download = """
    var csv = %s;

    var filename = 'testfile.txt';
    var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
    if (navigator.msSaveBlob) { // IE 10+
        navigator.msSaveBlob(blob, filename);
    } else {
        var link = document.createElement("a");
        if (link.download !== undefined) { // feature detection
            // Browsers that support HTML5 download attribute
            var url = URL.createObjectURL(blob);
            link.setAttribute("href", url);
            link.setAttribute("download", filename);
            link.style.visibility = 'hidden';
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        }
    }
    """ % open(filename,'r').readlines()

    display(Javascript(js_download))

interactive(children=(Dropdown(description='filename', options=('./testfile_old.txt', './testfile.txt'), value…